In [2]:
import json

In [3]:
jsonPath = "project_folder/breradesignweek_2019_events_20200122.json"
with open(jsonPath, mode='r', encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)

In [4]:
baseCompanies = [cc for cc in data
                 if cc['pacchetto_comunicazione'] == 'base']

plusCompanies = [cc for cc in data
                 if cc['pacchetto_comunicazione'] == 'plus']

sponsorCompanies = [cc for cc in data
                    if cc['pacchetto_comunicazione'] == 'sponsor']


In [50]:
len(sponsorCompanies)

52

In [5]:
espositori = [len(cc['espositori']) for cc in baseCompanies]

more_10 = sum(map(lambda x : x>10, espositori))
more_05 = sum(map(lambda x : 10>x>5, espositori))
less_03 = sum(map(lambda x : x<=5, espositori))

print('Companies: ' , len(espositori))
print('Companies with more than 10 espositori: ', more_10)
print('Companies with 6/10 espositori: ', more_05)
print('Companies with 5 espositori or less: ', less_03)

Companies:  118
Companies with more than 10 espositori:  3
Companies with 6/10 espositori:  4
Companies with 5 espositori or less:  110


In [6]:
miniEventi = [len(cc['mini-eventi']) for cc in plusCompanies]

more_10 = sum(map(lambda x : x>=10, miniEventi))
more_05 = sum(map(lambda x : 10>x>5, miniEventi))
more_03 = sum(map(lambda x : x>3, miniEventi))
less_03 = sum(map(lambda x : x<=3, miniEventi))

print('Companies: ' , len(miniEventi))
print('Companies with more than 10 miniEventi: ', more_10)
print('Companies with 6/10 miniEventi: ', more_05)
print('Companies with 5 miniEventi or less: ', less_03)

Companies:  38
Companies with more than 10 miniEventi:  4
Companies with 6/10 miniEventi:  30
Companies with 5 miniEventi or less:  4


In [7]:
vals = []
for cc in baseCompanies:
    for jj in cc['mini-eventi']:
        if jj['tipo_attivita'] not in vals:
            vals.append(jj['tipo_attivita'])

for cc in plusCompanies:
    for jj in cc['mini-eventi']:
        if jj['tipo_attivita'] not in vals:
            vals.append(jj['tipo_attivita'])


for cc in sponsorCompanies:
    for jj in cc['mini-eventi']:
        if jj['tipo_attivita'] not in vals:
            vals.append(jj['tipo_attivita'])

vals.sort()
vals

['altro',
 'conferenza',
 'esposizione',
 'festa_cocktail',
 'festa_riservata',
 'incontro',
 'premiazione',
 'presentazione',
 'press_preview',
 'show',
 'workshop']

In [8]:
import pandas as pd
N = 3
companiesList = []
for indexCompany, eachCompany in enumerate(baseCompanies):
    company = {}
    pageElement = f'{indexCompany%N+1:#02d}'
    company[pageElement + ' id'] = eachCompany['id']
    company[pageElement + ' id_fake'] = f'{indexCompany+1:#02d}'
    company[pageElement + ' titolo'] = eachCompany['titolo']['it']
    company[pageElement + ' desEN'] = eachCompany['descrizione_110']['it']
    companiesList.append(company)

def paginate(N, companies):
    '''
    Raggruppamento di elementi per pagina a seconda del valore scelto (da 1 a 4)
    Calcolo della mosca per selezione layout di pag. sinistra o destra.
    Butta fuori una pandas.dataFrame da esportare in csv (con separatore \t )
    '''
    paginatedCompanies = []
    pageCounter = 1
    for n in range(0, len(companies), N):
        pageCompanies = {}
        slice = companies[n:n+N]
        for company in slice:
            pageCompanies.update(company)
            
        # Inserisci mosca per selezione layout
        if pageCounter%2 == 1:
            pageCompanies['leftPage'] = '————————'
            pageCompanies['rightPage'] = ''
        else :
            pageCompanies['leftPage'] = ''
            pageCompanies['rightPage'] = '————————'

        pageCounter += 1
        paginatedCompanies.append(pageCompanies)

    df = pd.DataFrame(data=paginatedCompanies)
    return df

gigi = paginate(N, companiesList)
#.to_csv('test1.csv', sep='\t')


In [48]:
from os.path import exists, basename, splitext

MONTHS = ('gennaio','febbraio','marzo','aprile','maggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre')
PROJECT_FOLDER = "/Users/federicosantarini/Documents/GitHub/Automazione-Brera/project_folder"

for eachCompany  in plusCompanies:
#eachCompany = plusCompanies[1]
    title = eachCompany['titolo']['it']
    companyFolder = '/'.join([PROJECT_FOLDER, 'build', 'plus', f"{eachCompany['id']:#03d}_{title[:16]}"])
    logoName = basename(eachCompany['logo_azienda_file'])
    logoPath = '@' + companyFolder + '/' + logoName
    
        


SyntaxError: invalid syntax (<ipython-input-48-840996aafaca>, line 11)

In [10]:
import pandas as pd
from utils import groupSameDateAndTime, groupNonExhibitions

MONTHS = ('gennaio','febbraio','marzo','aprile','maggio','giugno','luglio','agosto','settembre','ottobre','novembre','dicembre')
N = 3

companiesList = []

def buildBaseTree(companies):
    for indexCompany, eachCompany in enumerate(companies):
        company = {}
        pageElement = f'{indexCompany%N+1:#02d} '

        # Titolo
        titolo = eachCompany['titolo']['it']
        company[pageElement + 'Titolo'] = titolo

        # Index sequenziale
        '''
        qui il problema sarà automatizzare la sequenzialità fra i vari pacchetti di comunicazione
        '''

        # Espositori (nome)
        exhibitors = [eachExhibitor['nome'] for eachExhibitor in eachCompany['espositori']]
        company[pageElement + 'Espositori'] = ', '.join(exhibitors)

        # Location (nome)
        locationName = eachCompany['location']['nome']
        company[pageElement + 'Nome Location'] = locationName

        # Location (indirizzo)
        locationAddress = eachCompany['location']['indirizzo']
        company[pageElement + 'Indirizzo Location'] = locationAddress

        # Descrizione 110 ita
        desIt = eachCompany['descrizione_110']['it']
        company[pageElement + 'Descrizione 110 ita'] = desIt

        # Descrizione 110 eng
        desEn = eachCompany['descrizione_110']['en']
        company[pageElement + 'Descrizione 110 eng'] = desEn

        # Esposizione (date, orari inizio/fine)
        exhibitions = [ev for ev in eachCompany['mini-eventi'] if ev['tipo_attivita']=='esposizione']
        mainEventDatesTimes = []
        for timespan, dates in groupSameDateAndTime(exhibitions).items():
            days = '/'.join([f'{dd.day:#02d}' for dd in dates])
            month = MONTHS[dates[0].month-1]
            mainEventDatesTimes.append(f'{days} {month} {timespan}')

        company[pageElement + 'Esposizione'] = ' / '.join(mainEventDatesTimes)

        # Mini eventi (attività, data, ora inizio/fine)
        nonExhibitionLenght = len([ev for ev in eachCompany['mini-eventi'] if ev['tipo_attivita']!='esposizione'])
        nonExhibition = groupNonExhibitions(eachCompany['mini-eventi'])
        secondaryEventDatesTimes = []
        if nonExhibition:
            nonExhibitionCounter = 0
            for eachKind, events in nonExhibition.items():
                kindRepr = eachKind.replace('_', ' ').title()
                groupedEvents = groupSameDateAndTime(events)
                for timespan, dates in groupedEvents.items():
                    days = '/'.join([f'{dd.day:#02d}' for dd in dates])
                    month = MONTHS[dates[0].month-1]
                    if nonExhibitionCounter != nonExhibitionLenght-1:
                        timespan = f'{timespan} |'
                    nonExhibitionCounter += 1
                    secondaryEventDatesTimes.append(f'{kindRepr} {days} {month} {timespan}')
        company[pageElement + 'mini-eventi'] = ' '.join(secondaryEventDatesTimes)

        # Append company data to companies list
        companiesList.append(company)
    print(companiesList[1])
